In [6]:
#氣象開放平台開發指南 
#https://opendata.cwb.gov.tw/devManual/insrtuction

#自動氣象站-氣象觀測資料
#https://opendata.cwb.gov.tw/fileapi/v1/opendataapi/O-A0001-001?Authorization=rdec-key-123-45678-011121314&format=JSON
#urllib3說明
#https://urllib3.readthedocs.io/

#ssl解決方案
#https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
    
import urllib3
import certifi
import json

#使用ultlib3, certifi, json下載資料和解析資料
http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',ca_certs=certifi.where())
r = http.request('GET', 'https://opendata.cwb.gov.tw/fileapi/v1/opendataapi/O-A0001-001?Authorization=rdec-key-123-45678-011121314&format=JSON')

if r.status == 200:
    downloadData=json.loads(r.data.decode('utf-8'))
else:
    print('下載錯誤')

#轉換成要使用的資料
usefulData = [];
opendata = downloadData['cwbopendata']['location']
for location in opendata:
    oneLocation = {}
    oneLocation['縣市']=location['parameter'][0]['parameterValue']
    oneLocation['區域']=location['parameter'][2]['parameterValue']
    oneLocation['時間']=location['time']['obsTime']
    oneLocation['溫度']=float(location['weatherElement'][3]['elementValue']['value'])
    usefulData.append(oneLocation)


In [7]:
import numpy as np
import pandas as pd
temperature = pd.DataFrame(usefulData)
temperature.head(20)

,縣市,區域,時間,溫度
0,臺北市,南港區,2020-10-11T16:00:00+08:00,25.3
1,雲林縣,臺西鄉,2020-10-11T16:00:00+08:00,27.0
2,臺南市,七股區,2020-10-11T16:00:00+08:00,27.4
3,臺東縣,成功鎮,2020-10-11T16:00:00+08:00,28.6
4,南投縣,南投市,2020-10-11T16:00:00+08:00,30.1
5,高雄市,茂林區,2020-10-11T16:00:00+08:00,27.6
6,連江縣,莒光鄉,2020-10-11T16:00:00+08:00,-99.0
7,澎湖縣,西嶼鄉,2020-10-11T16:00:00+08:00,23.8
8,澎湖縣,望安鄉,2020-10-11T16:00:00+08:00,25.3
9,金門縣,金沙鎮,2020-10-11T16:00:00+08:00,26.9


In [8]:
#sqlite 資料庫
import sqlite3
query = """
CREATE TABLE 氣象觀測資料(縣市 VARCHAR(20), 區域 VARCHAR(20), 時間 VARCHAR(30), 溫度 REAL);
"""

#建立方法只能建立一次，無法覆蓋原來檔案
con = sqlite3.connect('氣象觀測資料.sqlite')
con.execute(query)
con.commit()

In [9]:
for one in usefulData:    
    stmt = "INSERT INTO 氣象觀測資料 VALUES(?,?,?,?)"
    con.execute(stmt,(one['縣市'],one['區域'],one['時間'],one['溫度']))
    con.commit()
con.close()